In [1]:
!pip install catboost==0.20.2

     |████████████████████████████████| 63.9MB 61kB/s 


#DTC Prediction

# Catboost DTC prediction

In [0]:
import pandas as pd
import numpy as np
import xgboost as xgb
import catboost as cb
import lightgbm as lgbm
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math
import scipy.stats as st
%matplotlib inline
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

### Reading the different Data made available

In [0]:
#Now let's import and put the train and test datasets in  pandas dataframe

#train = pd.read_csv('/content/drive/My Drive/FACO/PDDA_COMPETITION/pdda_train.csv')
#test = pd.read_csv('/content/drive/My Drive/FACO/PDDA_COMPETITION/pdda_train.csv')
#valid = pd.read_csv("/content/real_result_20perc.csv")


df1 = pd.read_csv('/content/drive/My Drive/FACO/PDDA_COMPETITION/pdda_train.csv')
df2 = pd.read_csv('/content/drive/My Drive/FACO/PDDA_COMPETITION/pdda_test.csv')
valid = pd.read_csv("/content/drive/My Drive/FACO/PDDA_COMPETITION/real_result_20perc.csv")

### Replacing inconsistent entries in train-set with NAN

In [0]:
# remove all rows that contains missing value
df1.replace(['-999', -999], np.nan, inplace=True)


df2.replace(['-999', -999], np.nan, inplace=True)


In [0]:
df = df1.dropna()

In [0]:
X=df.drop(['DTC',"DTS",],axis =1)
y = df.DTC

In [0]:
test = valid.drop(['DTC','DTS'],axis=1)

### Training CatBoost Regressor for DTC prediction

In [0]:
from catboost import CatBoostRegressor
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
fold=KFold(n_splits=20)#15#5#10
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m2  = CatBoostRegressor( n_estimators=500,eval_metric='RMSE',learning_rate=0.2, random_seed= 1234, use_best_model=True,)
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=200,verbose=100,)#erly100
    preds=m2.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    errcb2.append(np.sqrt(mean_squared_error(y_test,preds)))
    p2 = m2.predict(test)
    y_pred_totcb2.append(p2)
np.mean(errcb2)

0:	learn: 18.2176992	test: 18.2176992	test1: 33.2242478	best: 33.2242478 (0)	total: 53.9ms	remaining: 26.9s
100:	learn: 3.0585720	test: 3.0585720	test1: 5.1619225	best: 5.1619225 (100)	total: 601ms	remaining: 2.37s
200:	learn: 2.6546443	test: 2.6546443	test1: 5.4954759	best: 5.1181044 (133)	total: 1.17s	remaining: 1.73s
300:	learn: 2.4011454	test: 2.4011454	test1: 5.7966948	best: 5.1181044 (133)	total: 1.74s	remaining: 1.15s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 5.118104436
bestIteration = 133

Shrink model to first 134 iterations.
err:  5.118104449348533
0:	learn: 18.1427686	test: 18.1427686	test1: 33.9201538	best: 33.9201538 (0)	total: 6.27ms	remaining: 3.13s
100:	learn: 3.0072758	test: 3.0072758	test1: 3.5390574	best: 3.5270374 (88)	total: 577ms	remaining: 2.28s
200:	learn: 2.6080030	test: 2.6080030	test1: 3.4603170	best: 3.4603170 (200)	total: 1.14s	remaining: 1.69s
300:	learn: 2.3726923	test: 2.3726923	test1: 3.5489890	best: 3.4599329 (201)	total: 1.69

4.365247153145564

### Checking Catboost Regressor Performance on Blind Validation set provided

In [0]:
cat_DTC_preds = np.mean(y_pred_totcb2, 0)

In [0]:
real_DTC_values = valid['DTC']

In [0]:
print('Root Mean Square Error is:', '{:.5f}'.format(np.sqrt(mean_squared_error(real_DTC_values, cat_DTC_preds))))

Root Mean Square Error is: 4.29563


# Making FINAL DTC Prediction ON Competition test DATA

In [0]:
df2.describe()

CAL           CNC            GR           HRD           HRM  \
count  11088.000000  11088.000000  11088.000000  11088.000000  11088.000000   
mean       8.634049      0.158501     28.966414      4.028372    106.752210   
std        0.044064      0.091298     43.648163      7.198112   2374.620246   
min        8.500000      0.009800      0.852000      0.083900      0.102700   
25%        8.625000      0.091700      8.449450      1.760775      1.865575   
50%        8.625000      0.128300     18.168650      2.758900      3.176950   
75%        8.671900      0.213500     36.470725      4.539675      5.034600   
max        8.875000      0.559400   1124.442000    202.233000  62290.769500   

                 PE          ZDEN  
count  11088.000000  11088.000000  
mean       7.353522      2.475884  
std        1.239075      0.149440  
min        4.760800      2.033400  
25%        6.528075      2.378175  
50%        7.881700      2.533600  
75%        8.308325      2.581800  
max       13.840700      3.024800

In [0]:
results = pd.DataFrame(columns=['DTC','DTS'])

In [0]:
y_pred_totcb1=[]

p1 = m2.predict(df2)
y_pred_totcb1.append(p1)

In [0]:
sub_cat_dtc_pred =  np.mean(y_pred_totcb1, 0)

results['DTC'] = sub_cat_dtc_pred

results.head()

DTC  DTS
0  102.714747  NaN
1  101.761868  NaN
2  103.348231  NaN
3  103.414831  NaN
4  104.958243  NaN

# Making LGBM DTS Prediction

### DTS Feature Engr Part

### Creation of More Features for DTS Prediction

In [0]:
gr_denom = df1.GR.max() - df1.GR.min()
df1['V_sh'] = (df1.GR - df1.GR.min())/ gr_denom

Saltwater_desnsity = 2.032
Salt_ft = 1.1
por_denom = Saltwater_desnsity - Salt_ft
df1['Por_est1'] = (Saltwater_desnsity - df1.ZDEN) / por_denom

Por_Factor = ( (df1.CNC ** 2) + (df1.ZDEN ** 2) ) / 2
df1['Por_est2'] = np.sqrt(Por_Factor)

df1['GZB'] = df1.ZDEN - df1.CNC

Hydrogen_mass = 1.00784
df1['Proton'] = Hydrogen_mass - df1.CNC

water_factor = ( df1.HRM / df1.HRD )
root = (1 / df1.CNC)
df1['Water_Sat'] = (water_factor ** root)


### Apply same method of Feature Creation to Blind Validation Data for Model Performance checkking or testing

In [0]:
valid1 = valid 

#V_SH
gr_denom = df1.GR.max() - df1.GR.min()
valid1['V_sh'] = (valid1.GR - df1.GR.min())/ gr_denom

#Porosity_Estimate1
Saltwater_desnsity = 2.032
Salt_ft = 1.1
por_denom = Saltwater_desnsity - Salt_ft
valid1['Por_est1'] = (Saltwater_desnsity - valid1.ZDEN) / por_denom

#Porosity_Estimate
Por_Factor = ( (valid.CNC ** 2) + (valid1.ZDEN ** 2) ) / 2
valid1['Por_est2'] = np.sqrt(Por_Factor)


#Gas_Zone_Boundry
valid1['GZB'] = valid1.ZDEN - valid1.CNC

#Proton_Reading
Hydrogen_mass = 1.00784
valid1['Proton'] = Hydrogen_mass - valid1.CNC

#Water_Saturation_Level
water_factor = ( valid1.HRM / valid1.HRD )
root = (1 / valid1.CNC)
valid1['Water_Sat'] = (water_factor ** root)

#### Dropping all Null Values and scaling of outliers in train set and validation set

In [0]:
df = df1.dropna()

In [0]:
train = df.drop(['DTC'],axis=1)
train['CNC'] = np.where((train.CNC >= 0.7),0.7,train.CNC)
train['HRM'] = np.where((train.HRM >= 10),10,train.HRM)
train['HRD'] = np.where((train.HRD >= 10),10,train.HRD)
train['GR'] = np.where((train.GR >= 150),150,train.GR)

In [0]:
valid1['CNC'] = np.where((valid1.CNC >= 0.7),0.7,valid1.CNC)
valid1['HRM'] = np.where((valid1.HRM >= 10),10,valid1.HRM)
valid1['HRD'] = np.where((valid1.HRD >= 10),10,valid1.HRD)
valid1['GR'] = np.where((valid1.GR >= 150),150,valid1.GR)

### Importing and Installing necessary packages for LGBM DTS Training and Prediction

In [0]:
import pandas as pd
import numpy as np
import xgboost as xgb
import catboost as cb
import lightgbm as lgbm
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math
import scipy.stats as st
from sklearn.preprocessing import RobustScaler, LabelEncoder, StandardScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.pipeline import make_pipeline
from scipy.special import boxcox1p


%matplotlib inline
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)


In [0]:
train_x = train.drop(['DTS','V_sh','Por_est1','Por_est2','GZB','Proton',],axis=1)
train_y = train['DTS']
params = {"max_depth": [5,10, 15],
         "learning_rate": [0.01, 0.05, 0.1,],
         "n_estimators": [100, 250,300,500,1000]}
params_cb = {'depth': [5, 10, 15],
            'learning_rate': [0.01, 0.05, 0.1],
            'iterations': [100, 250]}
test_y = valid['DTS']
test_x = valid1.drop(['DTC', 'DTS','V_sh','Por_est1','Por_est2','GZB','Proton',], axis=1)
n_splits = 5
max_train_size = len(train_x) // (n_splits+1)

In [0]:
%%time
model_lg = lgbm.LGBMRegressor()
grid_search_lg = GridSearchCV(model_lg, param_grid=params, cv=TimeSeriesSplit(n_splits=n_splits, max_train_size=max_train_size), verbose=1, n_jobs=-1)
grid_search_lg.fit(train_x, train_y)

Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   20.8s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  1.6min


CPU times: user 1.58 s, sys: 144 ms, total: 1.72 s
Wall time: 1min 52s


[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:  1.9min finished


### Checking Performance of LGBM regressor on Blind Validation data Provided

In [0]:
predict_DTS_lg1 = grid_search_lg.predict(test_x)
print('LightGBM RMSE Score: {}'.format(math.sqrt(mean_squared_error(test_y, predict_DTS_lg1))))
print("BEST PARAMETERS: " + str(grid_search_lg.best_params_))

LightGBM RMSE Score: 23.463127689180713
BEST PARAMETERS: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}


# Apply same Feature Engr to Submission Test DATA and Making DTS Prediction

In [0]:
valid1 = df2

#V_SH
gr_denom = df1.GR.max() - df1.GR.min()
valid1['V_sh'] = (valid1.GR - df1.GR.min())/ gr_denom

#Porosity_Estimate1
Saltwater_desnsity = 2.032
Salt_ft = 1.1
por_denom = Saltwater_desnsity - Salt_ft
valid1['Por_est1'] = (Saltwater_desnsity - valid1.ZDEN) / por_denom

#Porosity_Estimate
Por_Factor = ( (valid.CNC ** 2) + (valid1.ZDEN ** 2) ) / 2
valid1['Por_est2'] = np.sqrt(Por_Factor)


#Gas_Zone_Boundry
valid1['GZB'] = valid1.ZDEN - valid1.CNC

#Proton_Reading
Hydrogen_mass = 1.00784
valid1['Proton'] = Hydrogen_mass - valid1.CNC

#Water_Saturation_Level
water_factor = ( valid1.HRM / valid1.HRD )
root = (1 / valid1.CNC)
valid1['Water_Sat'] = (water_factor ** root)

In [0]:
valid1.describe()

CAL           CNC            GR           HRD           HRM  \
count  11088.000000  11088.000000  11088.000000  11088.000000  11088.000000   
mean       8.634049      0.158501     28.966414      4.028372    106.752210   
std        0.044064      0.091298     43.648163      7.198112   2374.620246   
min        8.500000      0.009800      0.852000      0.083900      0.102700   
25%        8.625000      0.091700      8.449450      1.760775      1.865575   
50%        8.625000      0.128300     18.168650      2.758900      3.176950   
75%        8.671900      0.213500     36.470725      4.539675      5.034600   
max        8.875000      0.559400   1124.442000    202.233000  62290.769500   

                 PE          ZDEN          V_sh      Por_est1     Por_est2  \
count  11088.000000  11088.000000  11088.000000  11088.000000  2217.000000   
mean       7.353522      2.475884      0.019799     -0.476270     1.607212   
std        1.239075      0.149440      0.029685      0.160343     0.073189   
min        4.760800      2.033400      0.000679     -1.065236     1.476368   
25%        6.528075      2.378175      0.005846     -0.589914     1.550657   
50%        7.881700      2.533600      0.012456     -0.538197     1.588750   
75%        8.308325      2.581800      0.024903     -0.371432     1.651959   
max       13.840700      3.024800      0.764818     -0.001502     1.820389   

                GZB        Proton     Water_Sat  
count  11088.000000  11088.000000  1.108800e+04  
mean       2.317383      0.849339  4.416358e+35  
std        0.223739      0.091298  4.650138e+37  
min        1.486300      0.448440  1.721187e-03  
25%        2.143825      0.794340  1.110878e+00  
50%        2.399500      0.879540  2.142858e+00  
75%        2.486800      0.916140  3.894208e+00  
max        2.685700      0.998040  4.896576e+39

In [0]:
test = valid1.drop(['V_sh','Por_est1','Por_est2','GZB','Proton',], axis=1)

In [0]:
y_pred_totcb3=[]

In [0]:
p3 = grid_search_lg.predict(test)
y_pred_totcb3.append(p3)

In [0]:
sub_lgb_dts_pred =  np.mean(y_pred_totcb3, 0)

In [0]:
results['DTS'] = sub_lgb_dts_pred

In [0]:
results.head()

DTC         DTS
0  102.714747  218.134730
1  101.761868  218.134730
2  103.348231  224.046092
3  103.414831  225.077764
4  104.958243  224.275838

In [0]:
results.to_csv('TEAM_TRIUMPHANT_submission_02.csv', index=False)